In [1]:
import findspark as fs
import os

data_path=os.path.dirname("D://Bigdata Tutorials//data//retail_db//")

fs.init()

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ITVERSITY').master('local').getOrCreate()
sc=spark.sparkContext
sc

<SparkContext master=local appName=ITVERSITY>

In [2]:
orderItems=sc.textFile(os.path.join(data_path,"order_items"))

In [3]:
int(orderItems.first().split(",")[1])

1

In [4]:
float(orderItems.first().split(",")[4])

299.98

## ITVERSITY CCA-175 Practice

In [5]:
orderItemsMap=orderItems.map(lambda x:(int(x.split(",")[1]),float(x.split(",")[4])))

In [6]:
revenuePerOrder=orderItemsMap.reduceByKey(lambda agg,x: agg+x)

In [7]:
for i in revenuePerOrder.take(10): print(i)

(1, 299.98)
(2, 579.98)
(4, 699.85)
(5, 1129.8600000000001)
(7, 579.9200000000001)
(8, 729.8399999999999)
(9, 599.96)
(10, 651.9200000000001)
(11, 919.79)
(12, 1299.8700000000001)


### So we have plucked order number and total revenue per order.

* This is done by using map() and reduceByKey()